In [1]:
#r "nuget: Jinaga"


Installed Packages Jinaga, 0.10.0

In [2]:
using Jinaga;
using Jinaga.Http;

In [5]:
class TokenAuthenticationProvider : IHttpAuthenticationProvider
{
    private string token;

    public TokenAuthenticationProvider(string token)
    {
        this.token = token;
    }

    public void SetRequestHeaders(System.Net.Http.Headers.HttpRequestHeaders headers)
    {
        headers.Add("Authorization", $"Bearer {token}");
    }

    public Task<bool> Reauthenticate()
    {
        return Task.FromResult(false);
    }
}

Create a file called `settings.json` and put the following settings into it:

```json
{
    "HttpEndpoint": "https://repdev.jinaga.com/....",
    "AuthorizationToken": "eyJh...."
}
```

In [11]:
#!value --from-file settings.json --name settings

In [12]:
#!share settings --from value

var parsedSettings = System.Text.Json.JsonSerializer.Deserialize<Dictionary<string, string>>(settings);

var jinagaClient = JinagaClient.Create(options =>
{
    options.HttpEndpoint = new Uri(parsedSettings["HttpEndpoint"]);
    options.HttpAuthenticationProvider = new TokenAuthenticationProvider(parsedSettings["AuthorizationToken"]);
});

In [8]:
[FactType("Blog.User.Name")]
public partial record UserName(User user, string value, UserName[] prior) { }

[FactType("Blog.Site")]
public partial record Site(User creator, DateTime createdAt) { }

[FactType("Blog.Site.Name")]
public partial record SiteName(Site site, string value, SiteName[] prior) { }

[FactType("Blog.Site.Domain")]
public partial record SiteDomain(Site site, string value, SiteDomain[] prior) { }

[FactType("Blog.Post")]
public partial record Post(Site site, User author, DateTime createdAt) { }

[FactType("Blog.Post.Title")]
public partial record PostTitle(Post post, string value, PostTitle[] prior) { }

[FactType("Blog.Post.Deleted")]
public partial record PostDeleted(Post post, DateTime deletedAt) { }

[FactType("Blog.Post.Publish")]
public partial record Publish(Post post, DateTime date) { }

In [13]:
var sitesForUser = Given<User>.Match((user, facts) =>
    from site in facts.OfType<Site>()
    where site.creator == user
    select new
    {
        id = jinagaClient.Hash(site),
        names =
            from name in facts.OfType<SiteName>()
            where name.site == site &&
                !facts.Any<SiteName>(next => next.prior.Contains(name))
            select name.value,
        domains =
            from domain in facts.OfType<SiteDomain>()
            where domain.site == site &&
                !facts.Any<SiteDomain>(next => next.prior.Contains(domain))
            select domain.value
    }
);

(var user, var profile) = await jinagaClient.Login();
var sites = await jinagaClient.Query(sitesForUser, user);

sites

index value 0 { id = +sYNBX9dkY7warmKFU+u5iesTSrk3a29AMNQbF6Hj3jElEd1TuCbs50ewh8bcpq88U9Tz+zNGfDLa3yTJxlhZQ==, names = System.Linq.Enumerable+<OfTypeIterator>d__66`1[System.String], domains = System.Linq.Enumerable+<OfTypeIterator>d__66`1[System.String] } id +sYNBX9dkY7warmKFU+u5iesTSrk3a29AMNQbF6Hj3jElEd1TuCbs50ewh8bcpq88U9Tz+zNGfDLa3yTJxlhZQ== names [ ] (values) [ ] domains [ ] (values) [ ] 1 { id = 8x5QVW0GXL0dEYQlg1J9uK8q231tm4imE9M+/jP2T+AXdBIxOM8Ag6vCQ2qnaYHlEPwZ9m3h68vA4r3kcUDrdg==, names = System.Linq.Enumerable+<OfTypeIterator>d__66`1[System.String], domains = System.Linq.Enumerable+<OfTypeIterator>d__66`1[System.String] } id 8x5QVW0GXL0dEYQlg1J9uK8q231tm4imE9M+/jP2T+AXdBIxOM8Ag6vCQ2qnaYHlEPwZ9m3h68vA4r3kcUDrdg== names [ Michael Perry Dot Net ] (values) [ Michael Perry Dot Net ] domains [ michaelperry.net ] (values) [ michaelperry.net ]